<h1>0. Packages</h1>

In [97]:
import numpy as np
import re
import skyfield.sgp4lib as spg4
import matplotlib 
import PyQt5
import generate_debris as gd

matplotlib.use('Agg')
import matplotlib.pyplot as plt
from scipy.stats import uniform
from scipy.stats import norm
import scipy.stats as stats
from enum import Enum    
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
from itertools import product, combinations
from matplotlib import cm
from scipy import integrate

%matplotlib qt5

debris_category = Enum('Category', 'rb sc soc')

<h1>1. Data Structure</h1>

<h3>1.1 Satellite Structure</h3>

Creating an object to represent satellite from 3le.txt <br>
May end up using implementation from another package

<b>Data sources:<b><br>
https://www.space-track.org    
https://www.celestrak.com/NORAD/documentation/spacetrk.pdf

<b>Information about 3le/2le:</b><br>
https://en.wikipedia.org/wiki/Two-line_element_set#cite_note-nasahelp-12
https://spaceflight.nasa.gov/realdata/sightings/SSapplications/Post/JavaSSOP/SSOP_Help/tle_def.html

In [2]:
class two_line_element:
    
    def __init__(self, data):
        self.parse_data(data)
        
    def parse_data(self, satelite_data):
        self.title = satelite_data[0]
        line_one = satelite_data[1].split()
        line_two = satelite_data[2].split()

        self.catalog_number = re.search('[0-9]{1,5}', line_one[0]).group()
        self.classification = re.search('[UCS]', line_one[0]).group()

        international_designator = re.search('([0-9]{2})([0-9]{3})(\w{1,3})', line_one[1])
        self.launch_year = international_designator.group(1)
        self.launch_number = international_designator.group(2)
        self.launch_piece =  international_designator.group(3)

        epoch = re.search('([0-9]{2})(\d+\.\d+)', line_one[2])
        self.epoch_year = epoch.group(1)
        self.epoch_day = epoch.group(2)

        self.ballistic_coefficient = float(line_one[3])
        self.mean_motion_double_prime = line_one[4]
        self.BSTAR = line_one[5]

        self.inclination = line_two[1]
        self.right_ascension_of_ascending_node = line_two[2]
        self.eccentricity = line_two[3]
        self.argument_of_perigee = line_two[4]

        mean_anomaly = line_two[5]

<h3>Data structure implementation using spg4 from skyfield library<h3>

In [3]:
# Opening the .txt file
with open("3le.txt") as f:
    txt = f.read()
    
sat_lines = re.findall('(.*?)\n(.*?)\n(.*?)\n', txt)

# Preview of `sat_lines`, each element should be the three lines representing a satellite
print(sat_lines[0])

# Convert each group of 3 lines into a satelite object
def line_element_to_satellite(lines):
    title = lines[0]
    line1 = lines[1]
    line2 = lines[2]
    return spg4.EarthSatellite(line1, line2, name=title)

satellites = [line_element_to_satellite(lines) for lines in sat_lines]

# Verify data structure by finding element representing the ISS
iss_index = [sat.name for sat in satellites].index("0 ISS (ZARYA)")
iss = satellites[iss_index]

('0 VANGUARD 1', '1     5U 58002B   19352.86539842  .00000184  00000-0  20948-3 0  9994', '2     5  34.2610  67.7117 1845282 255.2293  83.7559 10.84789954185624')


<h3>1.2 Celestial Bodies</h3>


de421.bsp is a Ephemerides provided by JPL Horizon which has the calculated positions of celestial bodies within a certain time interval. de421 is commonly used due to its small size and its relativley up to date information

In [4]:
from skyfield.api import load, EarthSatellite
ts = load.timescale(builtin=True)

# Loading the data from de421.bsp using skyfield 
planets = load('de421.bsp')

# Finding the data about Earth
earth = planets["Earth"]

<h3>1.3 Celestial Bodies</h3>

Plotting the satellite data in a 3D view.<br>
Accidently broke my implementation so need to start with the below base and get it to plot the ISS again.

<b>Source:</b> https://space.stackexchange.com/questions/25958/how-can-i-plot-a-satellites-orbit-in-3d-from-a-tle-using-python-and-skyfield

<h1>2. Breakup Models</h1>

<h3>2.1 NASA breakup model</h3>

<b>Implementation:</b> (Found on page 207)<br>
https://www.researchgate.net/publication/295490674_Space_debris_cloud_evolution_in_Low_Earth_Orbit

<b>Alternate implementation:</b><br>
https://gitlab.obspm.fr/apetit/nasa-breakup-model/tree/master

<b>Information:</b><br>
https://digitalcommons.calpoly.edu/cgi/viewcontent.cgi?referer=https://en.wikipedia.org/&httpsredir=1&article=1094&context=theses


<b>Now implemented in file generate_debris.py</b><br>

<h3>2.2 NASA breakup model validation</h3>

In [ ]:
L_c = gd.characteristic_lengths(1000, 10, 10, True, debris_category.rb)

N_fragments_total = L_c.shape[0]
lambda_c = np.log10(L_c)
areas = gd.avg_area(L_c)

def create_log_bins(values, nbins=100):
    #return np.geomspace(values.min(), values.max(), nbins)
    bins = np.geomspace(values.min(), values.max(), nbins)
    a = bins[1]/bins[0]
    bins = np.concatenate([[bins[0]/a], bins,[bins[-1]*a]])
    return bins

# Validating L_c
h, b = np.histogram(L_c, bins=create_log_bins(L_c))
plt.figure(figsize=(5,5))
plt.xscale('log')
plt.xlabel(r'$L_{c}$')
plt.ylabel(r'$N_f$')
plt.plot((b[:-1] + b[1:])/2, h, '.-')
plt.show()

# Validating Areas
h, b = np.histogram(areas, bins=create_log_bins(areas))
plt.figure(figsize=(5,5))
plt.xscale('log')
plt.xlabel(r'$Areas$')
plt.ylabel(r'$N_f$')
plt.plot((b[:-1] + b[1:])/2, h, '.-')
plt.show()

# Validating Mass & Velocity
AM = np.array(gd.distribution_AM(lambda_c, gd.debris_category.rb))
masses = areas / 10**AM

h, b = np.histogram(masses, bins=create_log_bins(masses))
plt.figure(figsize=(5,5))
plt.xscale('log')
plt.xlabel(r'$Masses$')
plt.ylabel(r'$N_f$')
plt.gca().set_yticks([0, 2, 4, 6, 8])
plt.gca().set_xticks([1e-8, 1e-5, 1e-2, 10, 1e4])
plt.gca().set_xlim([1e-8,1e4])
plt.gca().set_ylim([0,8])
plt.grid()
plt.plot((b[:-1] + b[1:])/2, h/1e5, '.-')
plt.show()

# Validating Velocity
deltaV = np.array(gd.distribution_deltaV(AM, 10, True))


h, b = np.histogram(deltaV, bins=create_log_bins(deltaV))
plt.figure(figsize=(5,5))
plt.xscale('log')
plt.xlabel(r'$\delta V$')
plt.ylabel(r'$N_f$')
plt.gca().set_yticks([0, 2, 4, 6, 8])
plt.gca().set_xticks([1e-3, 1e-2, 1e-1, 1, 10])
plt.gca().set_xlim([1e-3,10])
plt.gca().set_ylim([0,8])
plt.grid()
plt.plot((b[:-1] + b[1:])/2, h/1e5, '.-')
plt.show()

<h3>2.3 and Onward will be alternate breakup models</h3>

<h1>3. Numerical Propagation</h1>

<h3>3.1 Modeling the Atmosphere</h3>

In [43]:
import pandas as pd

excel_file = "AtmosphericModelValues.xlsx"
earth_radius = 6378.0 #km
tabulated_values = pd.read_excel(excel_file, engine="openpyxl")
tabulated_values.head()

def atmosphere_density(altitude):
    
    bins = tabulated_values['Altitude Lower Bound (km)'].values
    base_altitude = tabulated_values['Base Altitude (km)'].values
    nominal_density = tabulated_values['Nominal Density (kg/m^3)'].values
    scale_height = tabulated_values['Scale Height (km)'].values
    i = np.digitize(altitude, bins) - 1
    return (nominal_density[i]*np.exp(-(altitude-base_altitude[i])/(scale_height[i])))

def scale_height(altitude):
    bins = tabulated_values['Altitude Lower Bound (km)'].values
    scale_height = tabulated_values['Scale Height (km)'].values
    i = np.digitize(altitude, bins) - 1
    return scale_height[i]


<h3>3.2 Calculating Orbital Debris Properties</h3>

In [220]:
from skyfield.api import load, EarthSatellite

ts = load.timescale(builtin=True)

t = ts.now()
geocentric = iss.at(t)
init_position = geocentric.position.km

iss_mass = 419700 # kg
projectile_mass  = 227 # kg

L_c = gd.characteristic_lengths(iss_mass, projectile_mass, 100, True, debris_category.soc)
lambda_c = np.log10(L_c)
areas = gd.avg_area(L_c)
AM = np.array(gd.distribution_AM(lambda_c, debris_category.soc))
masses = areas / 10**AM
deltaV = np.array(gd.distribution_deltaV(AM, 10, True))


<h3>3.3 Converting Cartesian to Keplerian</h3>

https://downloads.rene-schwarz.com/download/M002-Cartesian_State_Vectors_to_Keplerian_Orbit_Elements.pdf

In [221]:
from numpy.linalg import norm

grav_param_earth = 398600.4418 #km^3s^-2
init_position = geocentric.position.km

deb_positions = np.empty((len(AM), 3))
deb_positions[:, :] = init_position[None,:]

deb_velocities = gd.velocity_vectors(len(AM), geocentric.velocity.km_per_s, deltaV)

orbital_momentums = np.cross(deb_positions, deb_velocities)
unit_positions = np.divide(deb_positions, norm(deb_positions, axis=1)[:, None]) ## Double check
eccentricities = (np.cross(deb_velocities, orbital_momentums) / grav_param_earth) - unit_positions

In [222]:
ascending_node_direction = np.cross(np.array([0, 0, 1])[None, :], orbital_momentums)

dot_ep = np.sum(eccentricities*deb_positions, axis=1)
np.sum(eccentricities*deb_positions, axis=1)

true_anomaly = np.arccos( dot_ep / (norm(eccentricities, axis=1) * norm(deb_positions, axis=1)))
B = np.sum(deb_positions*deb_velocities, axis=1)<0
true_anomaly[B] = 2*np.pi - true_anomaly[B]


In [223]:
inclination = np.arccos(orbital_momentums[:, 2] / norm(orbital_momentums, axis=1))

eccentricities_mag = norm(eccentricities, axis = 1)
theta_ea = np.tan(true_anomaly/2) / np.sqrt((1 + eccentricities_mag)/(1 - eccentricities_mag))
eccentric_anomaly = 2*np.arctan(theta_ea)

In [224]:
# Longitude of ascending node, Omega
long_ascending_node = np.arccos(ascending_node_direction[:,0]/norm(ascending_node_direction, axis=1))
B = ascending_node_direction[:,1]<0
long_ascending_node[B] = 2*np.pi - long_ascending_node[B]

# Argument of periapsis, omega
print(ascending_node_direction.shape)
dot_ne = np.sum(ascending_node_direction*eccentricities, axis=1)
arg_peri = np.arccos(dot_ne / (norm(ascending_node_direction, axis=1)*norm(eccentricities, axis=1)))
B = eccentricities[:,2] < 0
arg_peri[B] = 2*np.pi - arg_peri[B]

# Mean anomaly
mean_anomaly = eccentric_anomaly - norm(eccentricities,axis=1)*np.sin(eccentric_anomaly)

# Semi major axis
a = 1 / ((2/norm(deb_positions,axis=1)) - ((norm(deb_velocities, axis=1)**2) / grav_param_earth))

#semi parameter
p = norm(orbital_momentums, axis=1)**2 / grav_param_earth

#Orbital period
T = 2 * np.pi * np.sqrt(a**3 / grav_param_earth)

(5274775, 3)


In [186]:
deb_positions.shape

(5274775, 3)

In [ ]:
print("---- Pos. and Vel. ---")
print("pos: ", deb_positions[0,:], "km")
print("vel: ", deb_velocities[0,:], "km/s")

print("---- Orbital Elemets ---")
print("a: ", a[0]) # RIGHT
print("e: ", eccentricities_mag[0]) #RIGHT
print("i: ", inclination[0]) #RIGHT
print("Omega: ", long_ascending_node[0]) #RIGHT
print("omega: ", arg_peri[0]) #RIGHT
print("M: ", mean_anomaly[0]) # UNABLE TO VERIFY
print("v: ", true_anomaly[0]) #RIGHT

<h3>3.4 Integrating Keplerian elements</h3>

In [225]:
## Shrink number of debris being used
keplerian_states = np.array([a, eccentricities_mag, inclination, long_ascending_node, arg_peri, mean_anomaly, true_anomaly])
indexes = np.random.default_rng().choice(len(a), size=10000, replace=False)


keplerian_states = keplerian_states[:, indexes]
masses = masses[indexes]
areas = areas[indexes]
deb_positions = deb_positions[indexes, :]
deb_velocities = deb_velocities[indexes, :]

<h3> Implementing drag </h3>

"where cd is the drag coefficient of the fragment, which is assumed to be constant and equal to 2.2 (Valdo)"

In [316]:
from scipy.special import iv

def de_dt_drag(t, e, a, A, M, r):
    """Returns differential equation modeling drag on each piece of debris
    Args:
        M (array): Mass
        e (array): Eccentricity
        A (array): Cross-sectionl Area
        a (array): Semi-Major Axis
        r (array): Position
    """
    c_d = 2.2 # Drag coefficient
    ecc_const_coef = (-(c_d * A) / M) # Used in all
    altitude = np.linalg.norm(r) - earth_radius
    atm_density = atmosphere_density(altitude)
    I_k = lambda k, e: iv(k, (a * e) / scale_height(altitude))
    print(altitude)
    if ((0.01 <= e) & (e<= 0.2)):
        return (ecc_const_coef * np.sqrt(grav_param_earth / a) * atm_density)
    elif (0.001 <= e) & (e<= 0.01):
        return (ecc_const_coef * np.sqrt(grav_param_earth / a) * atm_density * (I_k(1, e) + (e/2)*(I_k(0, e) + I_k(2, e))))
    else:
        return 0
    
#     de_dt = np.empty(len(a), dtype=object)
#     indexes_1 = np.where((0.01 <= e) & (e<= 0.2))
#     indexes_2 = np.where((0.001 <= e) & (e<= 0.01))
#     indexes_3 = np.where(e < 0.001)    
#     de_dt[indexes_1] = lambda e :(ecc_const_coef[indexes_1] * np.sqrt(grav_param_earth / a[indexes_1]) * atm_density)
#     de_dt[indexes_2] = lambda e :(ecc_const_coef[indexes_2] * np.sqrt(grav_param_earth / a[indexes_2]) * atm_density * (I_k(1, e) + (e/2)*(I_k(0, e) + I_k(2, e))))
#     de_dt[indexes_3] = lambda e :0
   
    
# def da_dt_drag(a, e, A, M, r):
#     """Returns differential equation modeling drag on each piece of debris
#     Args:
#         M (array): Mass
#         e (array): Eccentricity
#         A (array): Cross-sectionl Area
#         a (array): Semi-Major Axis
#         r (array): Position
#     """
#     c_d = 2.2 # Drag coefficient
#     semi_const_coef = (-(c_d * A) / M) # Used in all
#     altitude = np.linalg.norm(r, axis = 1) - earth_radius
#     atm_density = atmosphere_density(altitude)
#     I_k = lambda k: iv(k, (a * e) / scale_height(altitude))
#     da_dt = np.empty(len(a), dtype=object)
#     indexes_1 = np.where((0.01 <= e) & (e<= 0.2))
#     indexes_2 = np.where((0.001 <= e) & (e<= 0.01))
#     indexes_3 = np.where(e < 0.001)    
#     bessel_func = (I_k(0) + 2*e*I_k(1) + (3/4)*e**2*(I_k(0) + I_k(2)) + (e**3/4)*(3*I_k(1) + I_k(3)))
    
#     da_dt[indexes_1] = lambda e : (semi_const_coef * np.sqrt(grav_param_earth * a) * atm_density * bessel_func)[indexes_1]
#     da_dt[indexes_2] = lambda e :(semi_const_coef * np.sqrt(grav_param_earth * a) * atm_density * (I_k(0) + 2*e*I_k(1)))[indexes_2]
#     da_dt[indexes_3] = lambda e : (semi_const_coef * np.sqrt(grav_param_earth * a) * atm_density)[indexes_3]
#     return da_dt

# Only works for one at a time
def da_dt_drag(t, e, a, A, M, r):
    c_d = 2.2 # Drag coefficient
    semi_const_coef = (-(c_d * A) / M) # Used in all
    altitude = np.linalg.norm(r) - earth_radius
    print("Alt: ", altitude)
    atm_density = atmosphere_density(altitude)
    I_k = lambda k: iv(k, (a * e) / scale_height(altitude))
    bessel_func = (I_k(0) + 2*e*I_k(1) + (3/4)*e**2*(I_k(0) + I_k(2)) + (e**3/4)*(3*I_k(1) + I_k(3)))
    
    if ((0.01 <= e) & (e<= 0.2)):
        return (semi_const_coef * np.sqrt(grav_param_earth * a) * atm_density * bessel_func)
    elif (0.001 <= e) & (e<= 0.01):
        return (semi_const_coef * np.sqrt(grav_param_earth * a) * atm_density * (I_k(0) + 2*e*I_k(1)))
    else:
        return (semi_const_coef * np.sqrt(grav_param_earth * a) * atm_density)
        
    
#de_dt = de_dt_drag(keplerian_states[0, :], keplerian_states[1, :], areas, masses, deb_positions)
#da_dt = da_dt_drag(keplerian_states[0, 0], keplerian_states[1, 0], areas[0], masses[0], deb_positions[0, :])

In [269]:
deb_positions[0, :]

array([ 5564.49668025, -1342.38667182, -3674.49096184])

In [317]:
t1, t2, dt = 0, 60*60*600, 60*10
times      = np.arange(t1, t2, dt)
#output1 = integrate.solve_ivp(de_dt_drag, (t1, t2), [keplerian_states[1, 0]],t_eval = times, args = (keplerian_states[0, 0], areas[0], masses[0], deb_positions[0, :]), vectorized=True).y[0]
output2 = integrate.solve_ivp(da_dt_drag, (t1, t2), [keplerian_states[1, 0]],t_eval = times, args = (keplerian_states[0, 0], areas[0], masses[0], deb_positions[0, :]), vectorized=True).y[0]

Alt:  424.0224279072236
Alt:  424.0224279072236
Alt:  424.0224279072236
Alt:  424.0224279072236
Alt:  424.0224279072236
Alt:  424.0224279072236
Alt:  424.0224279072236
Alt:  424.0224279072236
Alt:  424.0224279072236
Alt:  424.0224279072236
Alt:  424.0224279072236
Alt:  424.0224279072236
Alt:  424.0224279072236
Alt:  424.0224279072236
Alt:  424.0224279072236
Alt:  424.0224279072236
Alt:  424.0224279072236
Alt:  424.0224279072236
Alt:  424.0224279072236
Alt:  424.0224279072236
Alt:  424.0224279072236
Alt:  424.0224279072236
Alt:  424.0224279072236
Alt:  424.0224279072236
Alt:  424.0224279072236
Alt:  424.0224279072236
Alt:  424.0224279072236
Alt:  424.0224279072236
Alt:  424.0224279072236
Alt:  424.0224279072236
Alt:  424.0224279072236
Alt:  424.0224279072236
Alt:  424.0224279072236
Alt:  424.0224279072236
Alt:  424.0224279072236
Alt:  424.0224279072236
Alt:  424.0224279072236
Alt:  424.0224279072236
Alt:  424.0224279072236
Alt:  424.0224279072236
Alt:  424.0224279072236
Alt:  424.022427

In [314]:
output.y[0]
plt.plot(output1, times)
plt.show()

In [ ]:
# Works but could be better
def EccAnom(ec,M): 
  #ec=eccentricity, M=mean anomaly,
    i=0
    delta=10e-10; 
    E = []
    if (ec.all()<0.8):
        E=M
    else:
        E= M + np.sin(M)
        
    F = E - ec*np.sin(E) - M
    while ((np.abs(F).all()>delta) and (i<50)):
        E = E - F/(1.0-(ec* np.cos(E) ))
        F = E - ec * np.sin(E) - M
        i += 1
    return E


In [ ]:
import gc

t1, t2, dt = 0, 60*60*6, 60*10
times      = np.arange(t1, t2, dt)

mean_anomlay_rate_of_change = np.sqrt(grav_param_earth/a**3)

Nd = len(mean_anomlay_rate_of_change)
Nt = len(times)

eccentric_animales_vs_time = np.empty(shape=(Nd, Nt), dtype=np.float32)
for i in range(Nt):
    mean_anm_temp

mean_anomalies_vs_time = mean_anomaly[:, None] + mean_anomlay_rate_of_change[:, None] * times[None, :]
mean_anomalies_vs_time = mean_anomalies_vs_time % (2*np.pi)

eccentric_anomalies_vs_time = EccAnom(eccentricities_mag[:, None], mean_anomalies_vs_time)
x1 = np.sqrt(1 + eccentricities_mag)[:, None]*np.sin(eccentric_anomalies_vs_time / 2)
x2 = np.sqrt(1 - eccentricities_mag)[:, None]*np.cos(eccentric_anomalies_vs_time / 2)
true_anomalies_vs_time = 2*np.arctan2(x1, x2) % (2*np.pi)

<h3>3.5 Converting Keplerian to Cartesian</h3>

https://downloads.rene-schwarz.com/download/M001-Keplerian_Orbit_Elements_to_Cartesian_State_Vectors.pdf

In [ ]:
import gc

def keplerian_to_cartesian(Omega, omega, i, p, v, e, mu=grav_param_earth):

    cos_Omega = np.cos(Omega)
    cos_omega = np.cos(omega)
    cos_i = np.cos(i)
    sin_Omega = np.sin(Omega)
    sin_omega = np.sin(omega)
    sin_i = np.sin(i)
    

    rot_mat = np.array(     [(cos_Omega*cos_omega - sin_Omega*sin_omega*cos_i, -cos_Omega*sin_omega - sin_Omega*cos_omega*cos_i, sin_Omega*sin_i),
                            (sin_Omega*cos_omega + cos_Omega*sin_omega*cos_i, -sin_Omega*sin_omega + cos_Omega*cos_omega*cos_i, -cos_Omega*sin_i),
                            (sin_omega*sin_i, cos_omega*sin_i, cos_i)    ],dtype = np.float32).T
    
    del cos_Omega, cos_omega, cos_i
    del sin_Omega, sin_omega, sin_i
    gc.collect()
    
    r_pqw = np.array(((p[:, None]*np.cos(v)) / (1 + e[:, None]*np.cos(v)),
                      (p[:, None]*np.sin(v)) / (1 + e[:, None]*np.cos(v)),
                     np.zeros((len(Omega), v.shape[1]))), dtype = np.float32).T
    v_pqw = np.array((-np.sqrt(mu / p)[:, None]*np.sin(v),
                      np.sqrt(mu / p)[:, None] * (e[:, None] + np.cos(v)),
                      np.zeros((len(Omega), v.shape[1]))), dtype = np.float32).T
    
    pos = np.sum(rot_mat*r_pqw[:, :,:,  None], axis=2)
    
    vel = np.sum(rot_mat*v_pqw[:, :,:,  None], axis=2)
    
    
    
    return np.array([pos, vel], dtype = np.float32)

cartesian_state = keplerian_to_cartesian(Omega = long_ascending_node, omega = arg_peri,i= inclination, p=p,v= true_anomalies_vs_time, e = eccentricities_mag)

# NEED TO ASK ABOUT INTEGRATION AND ARRAY FRIENDLY CONVERSION
<h3>3.6 Performing the integration </h3>

<h1>4. Visualization </h1>

In [ ]:
# Creating 3D plot showing orbital information
# earth_radius = 6378.0 #km

# def create_3D_plot():
#     theta = np.linspace(0,2*np.pi, 360)
#     r = (a*(1-eccentricities_mag**2))/(1+eccentricities_mag*np.cos(theta)[:, None])
#     plt.polar(theta, r)
#     plt.show()
# #     fig = plt.figure()
# #     ax = fig.add_subplot(111, projection="3d")
    
# #     # Plot central body
# #     _u, _v = np.mgrid[0:2*np.pi:20j, 0:np.pi:10j]
# #     _x = earth_radius*np.cos(_u)*np.sin(_v)
# #     _y = earth_radius*np.sin(_u)*np.sin(_v)
# #     _z = earth_radius*np.cos(_v)
# #     ax.plot_surface(_x, _y, _z, cmap=cm.hot)
    
# #     # Plot axis
# #     unit = earth_radius * 2
# #     x,y,z = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]
# #     u, v, w =[[unit, 0, 0], [0, unit, 0], [0, 0, unit]]
# #     ax.quiver(x,y,z, u, v, w, color='k')
    
# #     ax.set_xlim([-2.1*earth_radius, 2.1*earth_radius])
# #     ax.set_ylim([-2.1*earth_radius, 2.1*earth_radius])
# #     ax.set_zlim([-2.1*earth_radius, 2.1*earth_radius])
    
# #     # Plot debris
# #     posititions_subset = positions[:, :1, :]
# #     ax.scatter(posititions_subset[:, :, 0], posititions_subset[:, :, 1], posititions_subset[:, : ,2], 'k', label="Debris")    
# #     plt.show()
    
# create_3D_plot()

In [ ]:
import mpl_toolkits.mplot3d.axes3d as p3
from matplotlib.animation import FuncAnimation
from matplotlib.animation import FFMpegWriter

data = cartesian_state[0, :, :1000, :]

def animate_scatters(iteration, data, scatters):
    """
    Update the data held by the scatter plot and therefore animates it.
    Args:
        iteration (int): Current iteration of the animation
        data (list): List of the data positions at each iteration.
        scatters (list): List of all the scatters (One per element)
    Returns:
        list: List of scatters (One per element) with new coordinates
    """
    for i in range(data[0].shape[0]):
        scatters[i]._offsets3d = (data[iteration][i,0:1], data[iteration][i,1:2], data[iteration][i,2:])
        scatters[i].set_color
    return scatters

fig = plt.figure()
ax = p3.Axes3D(fig)

ax.set_xlim3d(-6378.0-1000, 6378.0+1000)
ax.set_xlabel('X (km)')

ax.set_ylim3d([-6378.0-1000, 6378.0+1000])
ax.set_ylabel('Y (km)')

ax.set_zlim3d([-6378.0-1000, 6378.0+1000])
ax.set_zlabel('Z (km)')

# Plot central body
# earth_radius = 6378.0 #km
# _u, _v = np.mgrid[0:2*np.pi:20j, 0:np.pi:10j]
# _x = earth_radius*np.cos(_u)*np.sin(_v)
# _y = earth_radius*np.sin(_u)*np.sin(_v)
# _z = earth_radius*np.cos(_v)
# ax.plot_wireframe(_x, _y, _z)


# Initialize scatters
scatters = [ ax.scatter(data[0][i,0:1], data[0][i,1:2], data[0][i,2:]) for i in range(data[0].shape[0]) ]

# # Number of iterations
# iterations = len(data)
# ani = FuncAnimation(fig, animate_scatters, iterations, fargs=(data, scatters),
#                                        interval=200, blit=False, repeat=False)

# writervideo = FFMpegWriter(fps=5) 
# ani.save("test.mp4", writer=writervideo)